In [0]:
data = [12,34,56,768,890]
rDD=sc.parallelize(data,5)
print(rDD)
rDD.glom().collect()

ParallelCollectionRDD[0] at readRDDFromInputStream at PythonRDD.scala:413
Out[1]: [[12], [34], [56], [768], [890]]

In [0]:
A=sc.parallelize([12,34,56,78],4)
A.collect()

Out[5]: [12, 34, 56, 78]

In [0]:
A.reduce(lambda x,y:x*y)

Out[7]: 1782144

In [0]:
rdd=sc.parallelize([5,3,1,2])
print("takeOrdered",rdd.takeOrdered(4,lambda x:1*x))
print("count",rdd.count())
print("take",rdd.take(4))

takeOrdered [1, 2, 3, 5]
count 4
take [5, 3, 1, 2]


In [0]:
rdd.saveAsTextFile("/FileStore/rdddata/") 

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-3144494923315353> in <module>
----> 1 rdd.saveAsTextFile("/FileStore/rdddata/")

/databricks/spark/python/pyspark/rdd.py in saveAsTextFile(self, path, compressionCodecClass)
   1860             self.ctx._jvm.PythonRDD.saveAsTextFileImpl(keyed._jrdd, path, compressionCodecClass)
   1861         else:
-> 1862             self.ctx._jvm.PythonRDD.saveAsTextFileImpl(keyed._jrdd, path)
   1863 
   1864     # Pair functions

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    115     def deco(*a, **kw):
    116      

In [0]:
ls -ltr /dbfs/FileStore/rdddata/

ls: cannot access '/dbfs/FileStore/rdddata/': No such file or directory


In [0]:
word=["this","is","a","cat","running","fast"]
wordRDD=sc.parallelize(word)
wordRDD.reduce(lambda x,y : x if len(x)<len(y) else y)

Out[19]: 'a'

In [0]:
B=sc.parallelize([1,3,5,8,9])
B.reduce(lambda x,y:y-x)

Out[21]: 4

In [0]:
A=sc.parallelize(range(10000))
print(A.getNumPartitions())
D=A.repartition(4)
print(D.getNumPartitions())

8
4


In [0]:
A=A.coalesce(3)
print(A.getNumPartitions())

3


In [0]:
A=sc.parallelize(range(10000),numSlices=12)
print(A.getNumPartitions())

12


In [0]:
#Narrow Transforms
#MAP
rdd=sc.parallelize([1,2,3,4])
rdd.map(lambda x:x*2).collect()

Out[42]: [2, 4, 6, 8]

In [0]:
#Fliter
rdd.filter(lambda x:x%2!=0).collect()

Out[43]: [1, 3]

In [0]:
#mapvalues
rdd_map=sc.parallelize({('1',100),('2',200),('3',300)},3)
print(rdd_map.glom().collect())
rdd_map.mapValues(lambda x:x+1).collect()

[[('1', 100)], [('3', 300)], [('2', 200)]]
Out[48]: [('1', 101), ('3', 301), ('2', 201)]

In [0]:
data=["there is dog in park. Sitting alone"]
rdd_fmap=sc.parallelize(data)
rdd_fmap.flatMap(lambda x:x.split(" ")).collect()

Out[76]: ['there', 'is', 'dog', 'in', 'park.', 'Sitting', 'alone']

In [0]:
num_data=sc.parallelize(range(1,9),3)
#num_data.glom().collect()
def f(iterator): yield sum(iterator)
num_data.mapPartitions(f).collect()

Out[89]: [3, 12, 21]

In [0]:
rdd = sc.parallelize([1, 2, 3, 4], 4)
def f(splitIndex, iterator): yield splitIndex
rdd.mapPartitionsWithIndex(f).collect()

Out[95]: [0, 1, 2, 3]

In [0]:
#Wide Transformations
#cogroup
kv=[(1,2.5),(2,4.0),(3,5.0)]
kw=[(1,3.5),(1,4.5),(2,5.5),(2,5.6),(3,7.9),(3,8.0)]
rdd_kv=sc.parallelize(kv)
rdd_kw=sc.parallelize(kw)

In [0]:
cogroup_rdd=rdd_kv.cogroup(rdd_kw)
cogroup_rdd.collect()
[(x, tuple(map(list, y))) for x, y in sorted(list(cogroup_rdd.collect()))]

Out[100]: [(1, ([2.5], [3.5, 4.5])), (2, ([4.0], [5.5, 5.6])), (3, ([5.0], [7.9, 8.0]))]

In [0]:
#groupWith()
w = sc.parallelize([('a',5),('b',3)])
x=sc.parallelize([('a',1),('b',2)])
y=sc.parallelize([('a',4)])
z=sc.parallelize([('b',5)])
[(x, tuple(map(list, y))) for x, y in sorted(list(w.groupWith(x, y, z).collect()))]

Out[8]: [('a', ([5], [1], [4], [])), ('b', ([3], [2], [], [5]))]

In [0]:
#join()
rdd1 = sc.parallelize([(1,2),(2,1),(2,2)])
rdd2 = sc.parallelize([(2,5),(3,1)])
print("join:",rdd1.join(rdd2).collect())

join: [(2, (1, 5)), (2, (2, 5))]


In [0]:
#leftouterjoin()
print("leftjoin",rdd1.leftOuterJoin(rdd2).collect())

leftjoin [(1, (2, None)), (2, (1, 5)), (2, (2, 5))]


In [0]:
#rightOuterjoin()
print("rightjoin",rdd1.rightOuterJoin(rdd2).collect())

rightjoin [(2, (1, 5)), (2, (2, 5)), (3, (None, 1))]


In [0]:
rdd=sc.parallelize([(1,2),(2,4),(2,6),(3,9),(3,8),(4,1)])
print("groupBYkey",rdd.groupByKey().mapValues(lambda x:[a+a for a in x]).collect())

groupBYkey [(1, [4]), (2, [8, 12]), (3, [18, 16]), (4, [2])]


In [0]:
print("reduceBykey",rdd.reduceByKey(lambda a,b:a+b).collect())

reduceBykey [(1, 2), (2, 10), (3, 17), (4, 1)]


In [0]:
#combineByKey
rdd_com=sc.parallelize([('a',100),('b',200),('c',300),('a',300),('b',400),('d',200)])
combinerdd=rdd_com.combineByKey(lambda value :(value,1),lambda x,value:(x[0]+value,x[1]+1),lambda x,y:(x[0]+y[0],x[1]+y[1]))
combinerdd.collect()

Out[29]: [('a', (400, 2)), ('b', (600, 2)), ('c', (300, 1)), ('d', (200, 1))]

In [0]:
aggre_rdd=rdd_com.aggregateByKey((0,0),lambda acc,value:(acc[0]+value,acc[1]+1),lambda acc,value:(acc[0]+value[0],acc[1]+value[1]))
aggre_rdd.collect()

Out[33]: [('a', (400, 2)), ('b', (600, 2)), ('c', (300, 1)), ('d', (200, 1))]